In [4]:
!pip install -q scipy==1.2.0
!pip install -q numpy
!pip install -q rawpy
!pip install -q Pillow
!pip install -q matplotlib

ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [1]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [0]:
!python download_models.py

In [9]:
!python train_Sony_mine.py

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


2020-03-29 17:03:47.576821: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-29 17:03:47.581510: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-03-29 17:03:47.582844: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2020-03-29 17:03:47.583169: I tensorflow/stream_executor/platform/defa

In [0]:
!python train_Sony_mine_for_drive.py

In [0]:
!python test_Sony_50.py

In [0]:
!ls

In [5]:
!pwd

/content/drive/My Drive/Bakalarka-Peter_Sipos/Implementation/Learning-to-See-in-the-Dark


In [3]:
%cd /content/drive/My Drive/Bakalarka-Peter_Sipos/Implementation/Learning-to-See-in-the-Dark

/content/drive/My Drive/Bakalarka-Peter_Sipos/Implementation/Learning-to-See-in-the-Dark


In [0]:
a = []
while(1):
    a.append('1')

In [0]:
!pip install -q tensorflow==2.2.0-rc1